<a href="https://colab.research.google.com/github/MarceloaRuiz/CoderGit34675/blob/master/demo/vibevoice_realtime_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VibeVoice-Realtime Colab — T4 Quickstart



## Step 1: Setup Environment

In [1]:
# Check for T4 GPU
import torch
if torch.cuda.is_available() and "T4" in torch.cuda.get_device_name(0):
    print("✅ T4 GPU detected")
else:
    print("""
    ⚠️ WARNING: T4 GPU not detected

    The recommended runtime for this Colab notebook is "T4 GPU".

    To change the runtime type:

        1. Click on "Runtime" in the top navigation menu
        2. Click on "Change runtime type"
        3. Select "T4 GPU"
        4. Click "OK" if a "Disconnect and delete runtime" window appears
        5. Click on "Save"

    """)

# Clone the VibeVoice repository
![ -d /content/VibeVoice ] || git clone --quiet --branch main --depth 1 https://github.com/microsoft/VibeVoice.git /content/VibeVoice
print("✅ Cloned VibeVoice repository")

# Install project dependencies
!uv pip --quiet install --system -e /content/VibeVoice
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared && chmod +x cloudflared
print("✅ Installed dependencies")

# Download model
from huggingface_hub import snapshot_download
snapshot_download("microsoft/VibeVoice-Realtime-0.5B", local_dir="/content/models/VibeVoice-Realtime-0.5B")
print("✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B")


✅ T4 GPU detected
✅ Cloned VibeVoice repository
✅ Installed dependencies


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B


[Optional] If the download exceeds 1 minute, it is probably stuck. You can: (1) interrupt the execution, (2) log in to Hugging Face, and (3) try download again.

In [2]:
from huggingface_hub import login
login()

In [3]:
snapshot_download("microsoft/VibeVoice-Realtime-0.5B", local_dir="/content/models/VibeVoice-Realtime-0.5B")
print("✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Downloaded model: microsoft/VibeVoice-Realtime-0.5B


## Step 2: Launch VibeVoice-Realtime Demo

In [5]:
import torch, soundfile as sf
from huggingface_hub import snapshot_download
from vibevoice.modular.modeling_vibevoice_streaming_inference import VibeVoiceStreamingForConditionalGenerationInference
from vibevoice.processor.vibevoice_streaming_processor import VibeVoiceStreamingProcessor

# Asegurar GPU
assert torch.cuda.is_available(), "No hay GPU activa. Runtime → Change runtime type → GPU"

MODEL_ID = "microsoft/VibeVoice-Realtime-0.5B"
LOCAL_DIR = "/content/models/VibeVoice-Realtime-0.5B"
SR = 24000

# Descargar si no está
model_path = snapshot_download(MODEL_ID, local_dir=LOCAL_DIR)
print("✅ Modelo en:", model_path)

print("Cargando processor...")
processor = VibeVoiceStreamingProcessor.from_pretrained(LOCAL_DIR)

print("Cargando modelo en GPU...")
model = VibeVoiceStreamingForConditionalGenerationInference.from_pretrained(
    LOCAL_DIR,
    device_map="cuda",
    torch_dtype=torch.float16,
    attn_implementation="sdpa",

).eval()

text = """No entrenás para ganar.
Entrenás… para no abandonar.
Cuando nadie te ve."""
inputs = processor(text=text.strip(), return_tensors="pt").to("cuda")

print("Generando audio...")
with torch.no_grad():
    audio = model.generate(**inputs, cfg_scale=1.3, do_sample=False, inference_steps=2)

wav = audio.squeeze().detach().cpu().numpy()
sf.write("reel_voice.wav", wav, SR)
print("✅ Listo: reel_voice.wav")


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Qwen2Tokenizer'. 
The class this function is called from is 'VibeVoiceTextTokenizerFast'.


✅ Modelo en: /content/models/VibeVoice-Realtime-0.5B
Cargando processor...
Cargando modelo en GPU...


TypeError: VibeVoiceStreamingProcessor.__call__() got an unexpected keyword argument 'text'

In [6]:
import os, glob, torch, numpy as np, soundfile as sf
from huggingface_hub import snapshot_download

from vibevoice.modular.modeling_vibevoice_streaming_inference import (
    VibeVoiceStreamingForConditionalGenerationInference,
)
from vibevoice.processor.vibevoice_streaming_processor import (
    VibeVoiceStreamingProcessor,
)
from vibevoice.modular.streamer import AudioStreamer

MODEL_ID = "microsoft/VibeVoice-Realtime-0.5B"
LOCAL_DIR = "/content/models/VibeVoice-Realtime-0.5B"
SR = 24000

# GPU
assert torch.cuda.is_available(), "Runtime → Change runtime type → GPU"
DEVICE = torch.device("cuda")

# Descargar modelo si falta
snapshot_download(MODEL_ID, local_dir=LOCAL_DIR)
print("✅ Modelo en:", LOCAL_DIR)

# 1) Processor
processor = VibeVoiceStreamingProcessor.from_pretrained(LOCAL_DIR)

# 2) Modelo (sin flash_attn)
model = VibeVoiceStreamingForConditionalGenerationInference.from_pretrained(
    LOCAL_DIR,
    device_map="cuda",
    torch_dtype=torch.float16,
    attn_implementation="sdpa",
).eval()

# 3) Cargar un preset de voz (.pt)
voice_dir = "/content/VibeVoice/voices/streaming_model"
pt_files = sorted(glob.glob(os.path.join(voice_dir, "*.pt")))
assert pt_files, f"No encontré presets .pt en {voice_dir}"

preset_path = pt_files[0]  # podés cambiar a otro archivo luego
print("🎙️ Voice preset:", os.path.basename(preset_path))

cached_prompt = torch.load(preset_path, map_location=DEVICE, weights_only=False)

# 4) Texto del reel (editá solo esto)
text = """No entrenás para ganar.
Entrenás… para no abandonar.
Cuando nadie te ve."""

# 5) Preparar inputs (como en el server)
processed = processor.process_input_with_cached_prompt(
    text=text.strip(),
    cached_prompt=cached_prompt,
    padding=True,
    return_tensors="pt",
    return_attention_mask=True,
)

inputs = {k: (v.to(DEVICE) if hasattr(v, "to") else v) for k, v in processed.items()}

# 6) Generar audio juntando chunks del streamer
audio_streamer = AudioStreamer(batch_size=1, stop_signal=None, timeout=None)

with torch.no_grad():
    _ = model.generate(
        **inputs,
        max_new_tokens=None,
        cfg_scale=1.3,
        tokenizer=processor.tokenizer,
        generation_config={"do_sample": False, "temperature": 1.0, "top_p": 1.0},
        audio_streamer=audio_streamer,
        stop_check_fn=lambda: False,
        verbose=False,
        refresh_negative=True,
        all_prefilled_outputs=None,   # no hace falta duplicar acá
    )

chunks = []
for ch in audio_streamer.get_stream(0):
    if torch.is_tensor(ch):
        ch = ch.detach().cpu().to(torch.float32).numpy()
    ch = np.asarray(ch, dtype=np.float32).reshape(-1)
    chunks.append(ch)

wav = np.concatenate(chunks) if chunks else np.zeros((0,), dtype=np.float32)
sf.write("reel_voice.wav", wav, SR)
print("✅ Listo: reel_voice.wav  (duración aprox:", round(len(wav)/SR, 2), "s )")


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Qwen2Tokenizer'. 
The class this function is called from is 'VibeVoiceTextTokenizerFast'.


✅ Modelo en: /content/models/VibeVoice-Realtime-0.5B


AssertionError: No encontré presets .pt en /content/VibeVoice/voices/streaming_model

In [7]:
!find /content/VibeVoice -name "*.pt"



/content/VibeVoice/demo/voices/streaming_model/sp-Spk0_woman.pt
/content/VibeVoice/demo/voices/streaming_model/pt-Spk1_man.pt
/content/VibeVoice/demo/voices/streaming_model/en-Emma_woman.pt
/content/VibeVoice/demo/voices/streaming_model/pl-Spk0_man.pt
/content/VibeVoice/demo/voices/streaming_model/de-Spk0_man.pt
/content/VibeVoice/demo/voices/streaming_model/in-Samuel_man.pt
/content/VibeVoice/demo/voices/streaming_model/it-Spk0_woman.pt
/content/VibeVoice/demo/voices/streaming_model/fr-Spk0_man.pt
/content/VibeVoice/demo/voices/streaming_model/de-Spk1_woman.pt
/content/VibeVoice/demo/voices/streaming_model/en-Davis_man.pt
/content/VibeVoice/demo/voices/streaming_model/pl-Spk1_woman.pt
/content/VibeVoice/demo/voices/streaming_model/nl-Spk1_woman.pt
/content/VibeVoice/demo/voices/streaming_model/kr-Spk0_woman.pt
/content/VibeVoice/demo/voices/streaming_model/en-Grace_woman.pt
/content/VibeVoice/demo/voices/streaming_model/jp-Spk1_woman.pt
/content/VibeVoice/demo/voices/streaming_model/i

In [12]:
import os, torch, numpy as np, soundfile as sf
from huggingface_hub import snapshot_download

from vibevoice.modular.modeling_vibevoice_streaming_inference import (
    VibeVoiceStreamingForConditionalGenerationInference,
)
from vibevoice.processor.vibevoice_streaming_processor import (
    VibeVoiceStreamingProcessor,
)
from vibevoice.modular.streamer import AudioStreamer

# --- Config ---
MODEL_ID = "microsoft/VibeVoice-Realtime-0.5B"
LOCAL_DIR = "/content/models/VibeVoice-Realtime-0.5B"
VOICE_DIR = "/content/VibeVoice/demo/voices/streaming_model"
VOICE_FILE = "sp-Spk1_man.pt"   # <- probá también "sp-Spk0_woman.pt"
OUT_WAV = "reel_voice.wav"
SR = 24000

assert torch.cuda.is_available(), "Activá GPU: Runtime → Change runtime type → GPU"
DEVICE = torch.device("cuda")

# --- Download model (cached) ---
snapshot_download(MODEL_ID, local_dir=LOCAL_DIR)
print("✅ Modelo en:", LOCAL_DIR)

# --- Load processor + model ---
print("Cargando processor...")
processor = VibeVoiceStreamingProcessor.from_pretrained(LOCAL_DIR)

print("Cargando modelo en GPU (sin flash_attn)...")
model = VibeVoiceStreamingForConditionalGenerationInference.from_pretrained(
    LOCAL_DIR,
    device_map="cuda",
    torch_dtype=torch.float32,
    attn_implementation="sdpa",   # <- evita flash_attn faltante
).eval()

# --- Load voice preset ---
preset_path = os.path.join(VOICE_DIR, VOICE_FILE)
assert os.path.exists(preset_path), f"No existe preset: {preset_path}"
print("🎙️ Voice preset:", preset_path)

cached_prompt = torch.load(preset_path, map_location=DEVICE, weights_only=False)

# --- Text (EDIT THIS) ---
text = """No le tengas miedo al lactato.
No es tu enemigo…
es un mensaje.

Cuando sentís que las piernas arden,
no es que estés fallando.
Es tu cuerpo diciendo:
estoy yendo más rápido de lo que puedo sostener… por ahora.

El lactato no aparece para frenarte.
Aparece para marcar un límite.
Y los límites… se entrenan.

Cada vez que te acercás a ese punto,
cada vez que respirás fuerte y decidís seguir,
tu cuerpo aprende a usar mejor esa energía.

No se trata de sufrir por sufrir.
Se trata de entender el esfuerzo.
De escuchar lo que el cuerpo dice
y entrenar con intención.

Porque mejorar no es evitar el lactato.
Mejorar…
es aprender a convivir con él
un poco más de tiempo."""

# --- Prepare inputs (streaming processor API) ---
processed = processor.process_input_with_cached_prompt(
    text=text.strip(),
    cached_prompt=cached_prompt,
    padding=True,
    return_tensors="pt",
    return_attention_mask=True,
)
inputs = {k: (v.to(DEVICE) if hasattr(v, "to") else v) for k, v in processed.items()}

# --- Generate audio via streamer ---
audio_streamer = AudioStreamer(batch_size=1, stop_signal=None, timeout=None)

with torch.no_grad():
    _ = model.generate(
        **inputs,
        max_new_tokens=None,
        cfg_scale=1.3,
        tokenizer=processor.tokenizer,
        generation_config={"do_sample": False, "temperature": 1.0, "top_p": 1.0},
        audio_streamer=audio_streamer,
        stop_check_fn=lambda: False,
        verbose=False,
        refresh_negative=True,
        all_prefilled_outputs=cached_prompt,

    )

chunks = []
for ch in audio_streamer.get_stream(0):
    if torch.is_tensor(ch):
        ch = ch.detach().cpu().to(torch.float32).numpy()
    ch = np.asarray(ch, dtype=np.float32).reshape(-1)
    chunks.append(ch)

wav = np.concatenate(chunks) if chunks else np.zeros((0,), dtype=np.float32)
sf.write(OUT_WAV, wav, SR)

print(f"✅ Generado: {OUT_WAV} | duración: {round(len(wav)/SR, 2)} s")


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Qwen2Tokenizer'. 
The class this function is called from is 'VibeVoiceTextTokenizerFast'.


✅ Modelo en: /content/models/VibeVoice-Realtime-0.5B
Cargando processor...
Cargando modelo en GPU (sin flash_attn)...
🎙️ Voice preset: /content/VibeVoice/demo/voices/streaming_model/sp-Spk1_man.pt



Prefilled 381 tokens, current step (381 / 8192):   5%|▍         | 381/8192 [00:00<?, ?it/s]
Prefilled 5 text tokens, generated 0 speech tokens, current step (386 / 8192):   5%|▍         | 386/8192 [00:00<00:01, 7581.89it/s]
Prefilled 5 text tokens, generated 0 speech tokens, current step (386 / 8192):   5%|▍         | 387/8192 [00:00<04:04, 31.94it/s]  
Prefilled 5 text tokens, generated 1 speech tokens, current step (387 / 8192):   5%|▍         | 387/8192 [00:00<04:04, 31.94it/s]
Prefilled 5 text tokens, generated 2 speech tokens, current step (388 / 8192):   5%|▍         | 388/8192 [00:00<04:04, 31.94it/s]
Prefilled 5 text tokens, generated 3 speech tokens, current step (389 / 8192):   5%|▍         | 389/8192 [00:00<04:04, 31.94it/s]
Prefilled 5 text tokens, generated 4 speech tokens, current step (390 / 8192):   5%|▍         | 390/8192 [00:00<04:04, 31.94it/s]
Prefilled 5 text tokens, generated 4 speech tokens, current step (390 / 8192):   5%|▍         | 391/8192 [00:00<12:06, 10.7

✅ Generado: reel_voice.wav | duración: 72.4 s


In [14]:
from google.colab import files
files.download("reel_voice.wav")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>